In [5]:
### Importing required python package
import pandas as pd
import numpy as np
import json          
from pyspark.sql import SparkSession 

### This code is for POC purpose. Dont deploy this in production environment without tuning further and 
### implement exception handling

In [6]:
import time
starttime = time.time()

from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2018-07-27 09:28:53'

In [7]:
## Define Increment Load or Full Load
loadtype = 'incrementalload'
#loadtype = 'fullload'
RUN_DATE = '20180531000000'

In [8]:
import datetime

# naive datetime
rundate = datetime.datetime.strptime(RUN_DATE, '%Y%m%d%H%M%S')

print("The entire load will happen upto {}".format(rundate))
print("and this is {}".format(loadtype))


The entire load will happen upto 2018-05-31 00:00:00
and this is incrementalload

In [9]:
### Create spark session 
### This steps required to run from jupyter
spark = SparkSession.builder.appName("Second SparkSession").getOrCreate()

In [10]:
# date_format( date_format( t1.flt_schd_dep_date  , 'yyyy-MM-dd HH.mm.ss.S'), 'yyyyMMddHHmmss') 
### Bringing the dataset for menu for future flight
spark_future_menu = spark.sql("select t1.flt_carrier_code, t1.flt_number as flight_number, CONCAT(SUBSTR(t1.flt_schd_dep_date,1,4) , SUBSTR(t1.flt_schd_dep_date,6,2) , SUBSTR(t1.flt_schd_dep_date,9,2),SUBSTR(t1.flt_schd_dep_date,12,2),SUBSTR(t1.flt_schd_dep_date,15,2) ) as flight_boarding_time, t1.flt_schd_dep_stn as board_point, t1.cabin_class_code, t1.service_category_code, t1.meal_service_name, t1.meal_service_description, t1.menu_name, t1.meal_type_code, t2.menu_id, t2.item_id , t2.item_code, t2.item_name, t2.category, t2.subcategory, t2.quantity, t2.cmsm_uom_name, t3.dishcode, t3.dishversion, t3.dishcategory, t3.dishsubcategory, t3.cabinclassname, t3.menucardname, t3.dishname, t3.is_m_level_boolean, t3.approved_boolean, t3.ek_sig_dish_boolean, t3.in_use_boolean, t3.cms_status, t3.cms_createdate, t3.effective_from, t3.effective_to, t3.lw_cal_dish_boolean, t3.menu_text_boolean from flight_menu_master t1 inner join ( select * from menu_itemid_dishcode_master where category = 'Main Course' ) as t2 on t1.menu_id = t2.menu_id inner join ( select * from meals_itemid_details_master where dishcategory = 'Main Course' ) as t3 on t2.item_id = t3.item_id where t1.flt_schd_dep_date > ( select from_unixtime(unix_timestamp(max(rundate),'yyyyMMddHHmmss' ) + 86400 ) from loadtable )") 

In [11]:
# spark_future_menu.printSchema()

In [12]:
### testing spark_flight data 
# print(spark_future_menu.count()) ### Testing result 57882 as of 6/14
# spark_future_menu.show()

In [13]:
### Bringing the dataset for spark_pax_flight_final
spark_pax_flight = spark.sql("select flight_number, flight_boarding_time, board_point, id, pax_id, pax_booking_status, destination, cabin_class, booked_class, travelled_class, SeatNumber, pax_type, pax_group_code, boarding_date, upgrade_indicator, ticket_class, handicapped, unaccompanied_minor, priority_pax, check_in_city, date_of_birth, nationality, gender, country_of_birth, country_of_residence,airline_designator,aircraft_type,class_configuration,crew_code,gate_number from spark_pax_flight_final")

In [14]:
# spark_pax_flight.printSchema()

In [15]:
### testing spark_pax data 
#print(spark_pax_flight.count())
#spark_pax_flight.show()

In [16]:
from pyspark.sql.functions import broadcast

spark_pax_flight_key  = [ "flight_number" , "flight_boarding_time" , "board_point" ]
spark_pax_flight = spark_pax_flight.join(broadcast(spark_future_menu), spark_pax_flight_key , "inner")

In [17]:
### testing spark_pax_flight data 
#print(spark_pax_flight.count())
#spark_pax_flight.show()

In [18]:
### Loading the pax and flights data into Hive table
spark_pax_flight.createOrReplaceTempView("spark_pax_flight")

if loadtype is "fullload" :
    print("loading spark_pax_flight_future_final table")
    sqlContext.sql("create table spark_pax_flight_future_final as select * from spark_pax_flight")
else:
    print("Dropping spark_pax_flight_future_final table")
    sqlContext.sql("drop table spark_pax_flight_future_final")
    print("loading spark_pax_flight_future_final table")
    sqlContext.sql("create table spark_pax_flight_future_final as select * from spark_pax_flight")    

Dropping spark_pax_flight_future_final table
DataFrame[]
loading spark_pax_flight_future_final table
DataFrame[]

In [19]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2018-07-27 09:30:10'

In [20]:
endtime = time.time()
diff = int ( endtime - starttime ) 
minutes, seconds = diff // 60, diff % 60

print("time taken: {} mins & {} secs ".format(minutes,seconds))

time taken: 1 mins & 17 secs